# Personal Music Collection Database

Time to gain some experience with SQL! In this project, I hope to both demonstrate and strengthen my knowledge of SQL by creating a database which contains information about my physical music collection. Although I do enjoy the ease of streaming services, there is something very special and nostalgic to me about owning physical media of the albums I love. In this project, let's use SQL to organize and understand my collections of records and cds.

> Note - This file primarily contains the creation of tables in my database, as well as functions for table insertion and querying. I provide documentation of these processes step-by-step, typically with a few examples of useage. I intend to create a separate file with more extensive usage of these functions.

# Table Creation

Below, I import sqlite3 and connect to/create a database for my personal music connection. Next, I create four tables within this database. The first is for each artist I have physical media from, and the table contains that artist's id and name. The next table I created is for the albums themselves, and the table contains information pertaining to each such as its title, the id of the artist featured on it, and the year it was created. My third table simply distinguishes format type, pairing an id with each type of format (cd, vinyl, etc). Lastly, my final table combines the previous tables, identifying the albums based on both artist and format. 

In [26]:
# Import SQL
import sqlite3

# Connect to the database (which, we are creating it, because it does not yet exist)
conn = sqlite3.connect('music_collection.db')
cursor = conn.cursor()

# Create a table for each artist I have music by
# In later adaptations we could extend this to create table of bands, and each person in said band (ie, Stevie Nicks is both a solo artist and in Fleetwood Mac)
cursor.execute("""
        CREATE TABLE IF NOT EXISTS Artists (
               id INTEGER PRIMARY KEY,
               name TEXT UNIQUE NOT NULL
        )
    """)

# Create a table for each album I own, given its title, artist, and release year
# In later adaptations, how can we consider features on albums? Multi-artist collaborations?
cursor.execute("""
        CREATE TABLE IF NOT EXISTS Albums (
               id INTEGER PRIMARY KEY,
               title TEXT UNIQUE NOT NULL,
               artist_id INTEGER NOT NULL,
               year INTEGER,
               FOREIGN KEY (artist_id) REFERENCES Artists(id),
               UNIQUE (title, artist_id)
        )
    """)

# Create a table for format types (because I own CDS, vinyl, and a few cassettes)
cursor.execute("""
        CREATE TABLE IF NOT EXISTS Formats (
               id INTEGER PRIMARY KEY,
               format_name TEXT UNIQUE NOT NULL
        )
    """)

# Create another table based on album/format type, combining the above information
cursor.execute("""
        CREATE TABLE IF NOT EXISTS FormattedAlbums (
               album_id INTEGER NOT NULL,
               format_id INTEGER NOT NULL,
               FOREIGN KEY (album_id) REFERENCES Albums(id),
               FOREIGN KEY (format_id) REFERENCES Formats(id),
               PRIMARY KEY (album_id, format_id)
        )
    """)

# Insertion Functions

Great! Now that we have these tables created, let's define some functions which will allow us to insert/query artists and albums.

To start, lets create functions which insert our values onto each table. Additionally, I have created a few functions which find the id associated to an artist, album, or format, which I use in later functions as needed. 

In [28]:
def insert_artist(name):
    """This function inserts a new artist into our Artists table."""
    cursor.execute("""INSERT OR IGNORE INTO Artists (name) VALUES (?)""", (name,))
    conn.commit()
    return cursor.lastrowid

def find_artist_id(name):
    cursor.execute("""SELECT id FROM Artists WHERE name = ?""", (name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    return None

def insert_album(title, artist_name, year):
    """This function inserts an album into our Albums table if it doesn't already exist."""
    artist_id = find_artist_id(artist_name)
    if not artist_id:
        artist_id = insert_artist(artist_name) # if for some reason the artist is not in our Artists table, add it.
    
    # Check if the album already exists
    cursor.execute("""SELECT id FROM Albums WHERE title = ? AND artist_id = ?""", (title, artist_id))
    result = cursor.fetchone()
    if result:
        print(f"Album '{title}' by '{artist_name}' already exists in the database.")
        return result[0]
    else:
        cursor.execute("""INSERT INTO Albums (title, artist_id, year) VALUES (?, ?, ?)""", (title, artist_id, year))
        conn.commit()
        return cursor.lastrowid

def find_album_id(title):
    cursor.execute("""SELECT id FROM Albums WHERE title = ?""", (title,))
    result = cursor.fetchone()
    if result:
        return result[0]
    return None

def load_format(format):
    """Insert any new possible format types of our media."""
    cursor.execute("""INSERT INTO Formats (format_name) VALUES (?)""", (format,))
    conn.commit()
    return cursor.lastrowid

def find_format_id(format):
    """We also need the id associated with each format type."""
    cursor.execute("""SELECT id from Formats WHERE format_name = ?""", (format,))
    format_id = cursor.fetchone()[0]
    return format_id

def insert_formatted_album(album_title, format_title):
    """Finally, we will use the above functions in order to insert an album also with its media type."""
    album_id = find_album_id(album_title)
    format_id = find_format_id(format_title)
    cursor.execute("""INSERT OR IGNORE INTO FormattedAlbums (album_id, format_id) VALUES (?, ?)""", (album_id, format_id))
    conn.commit()
    return cursor.lastrowid


# Query Functions

Next, lets create some functions which Query the tables in our database:

In [3]:
def query_artists():
    """Query all artists."""
    cursor.execute("SELECT * FROM Artists")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

def query_albums():
    """Query all albums."""
    cursor.execute("SELECT * FROM Albums")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

def query_formats():
    """Query all formats."""
    cursor.execute("SELECT * FROM Formats")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

def query_formatted_albums():
    """Query all formatted albums."""
    cursor.execute("""
    SELECT Albums.title, Artists.name, Albums.year, Formats.format_name
    FROM FormattedAlbums
    JOIN Albums ON FormattedAlbums.album_id = Albums.id
    JOIN Artists ON Albums.artist_id = Artists.id
    JOIN Formats ON FormattedAlbums.format_id = Formats.id
    """)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

After creating functions which both insert values into tables of our database, as well as query functions, we can demonstrate these functionalities by testing them. For example, let's add both Stevie Nicks and Fleetwood Mac to our Artists table. 

In [4]:
insert_artist('Stevie Nicks')
insert_artist('Fleetwood Mac')

0

In [5]:
query_artists()

(1, 'Stevie Nicks')
(2, 'Fleetwood Mac')
(3, 'Taylor Swift')


Next, let's insert an album:

In [6]:
insert_album('Bella Donna', 'Stevie Nicks', 1981)

Album 'Bella Donna' by 'Stevie Nicks' already exists in the database.


1

And then query the table:

In [7]:
query_albums()

(1, 'Bella Donna', 1, 1981)
(2, 'Tango In The Night', 2, 1987)
(3, 'Evermore', 3, 2020)
(4, 'Greatest Hits', 2, 1988)
(5, 'Penguin', 2, 1973)


I own Bella Donna on vinyl, so let's make sure that was added to our table of possible format types... 

In [8]:
query_formats()

(1, 'CD')
(2, 'vinyl')
(3, 'cassette')


Perfect. Lastly, lets add the album to our table which includes description of format:

In [9]:
insert_formatted_album('Bella Donna', 'vinyl')

0

And query this table...

In [10]:
query_formatted_albums()

('Tango In The Night', 'Fleetwood Mac', 1987, 'vinyl')
('Evermore', 'Taylor Swift', 2020, 'vinyl')
('Greatest Hits', 'Fleetwood Mac', 1988, 'vinyl')
('Penguin', 'Fleetwood Mac', 1973, 'vinyl')
('Tango In The Night', 'Fleetwood Mac', 1987, 'CD')
('Bella Donna', 'Stevie Nicks', 1981, 'vinyl')


Now that we have a functional system with Formatted Albums, we can further extend this project. For example, let's say that I give some albums away to a friend. It would be convenient to additionally have delete functions for tables where it seems applicable...

In [11]:
#def delete_album(album_name):
    #"""Delete an album from Albums."""
    #album_id = find_album_id(album_name)
    #cursor.execute("""DELETE FROM Albums WHERE id = ?""", (album_id,))

def delete_formatted_album(album_name, format_name):
    """Delete a formatted album from FormattedAlbums."""
    album_id = find_album_id(album_name)
    format_id = find_format_id(format_name)
    cursor.execute("""DELETE FROM FormattedAlbums WHERE album_id = ? AND format_id = ?""", (album_id, format_id))

Hypothetically, I loan my vinyl Bella Donna album to a friend: it leaves my posession, so it also leaves the formatted table...

In [12]:
delete_formatted_album('Bella Donna', 'vinyl')
query_formatted_albums()

('Tango In The Night', 'Fleetwood Mac', 1987, 'vinyl')
('Evermore', 'Taylor Swift', 2020, 'vinyl')
('Greatest Hits', 'Fleetwood Mac', 1988, 'vinyl')
('Penguin', 'Fleetwood Mac', 1973, 'vinyl')
('Tango In The Night', 'Fleetwood Mac', 1987, 'CD')


We now have a way to delete albums from my collection. To me, it makes sense to simply create 'delete_formatted_album' for functionality with little need to create a delete Artist or Album function. Having those on hand to work with our database may be nice and we can create them later if necessary, but functionally I feel as though we only truly need to keep an accurate record of which physical pieces of media I have in my posession. Getting rid of my CD does not mean the album or artist should necessarily be deleted. 

# Querying Based on Time Period

Next, we can create some functions which allow me to find trends in our database, and thus also allow me to organize my music collection in various ways. For example, what if I wanted to see which albums I have from various time periods? I love buying my current favorites, but I have also inherited a lot of vinyl from my parents. Let's create a function that allows me to see what music I have collected between various years:

In [13]:
def formatted_album_between_years(start_year, end_year):
    """Query formatted albums which were released between the specified start and end years."""
    cursor.execute(
        """SELECT Albums.title, Artists.name, Albums.year, Formats.format_name
            FROM FormattedAlbums
            JOIN Albums ON FormattedAlbums.album_id = Albums.id
            JOIN Artists ON Albums.artist_id = Artists.id
            JOIN Formats ON FormattedAlbums.format_id = Formats.id
            WHERE Albums.year BETWEEN ? AND ?""",
            (start_year, end_year) )
    between_albums = cursor.fetchall()
    return between_albums

Let's test this function by adding a few more albums back into our FormattedAlbums table.

In [14]:
# Bella Donna is still in our Albums table, now let's re-instate that I have it in vinyl... My friend gave it back!
insert_formatted_album('Bella Donna', 'vinyl')

# Let's put Tango In The Night into both tables as well
insert_album('Tango In The Night', 'Fleetwood Mac', 1987)
insert_formatted_album('Tango In The Night', 'vinyl')

# Lastly, for some comparison between time periods, let's add something more recent - Evermore by Taylor Swift, which I have on vinyl
insert_artist('Taylor Swift')
insert_album('Evermore', 'Taylor Swift', 2020)
insert_formatted_album('Evermore', 'vinyl')

Album 'Tango In The Night' by 'Fleetwood Mac' already exists in the database.
Album 'Evermore' by 'Taylor Swift' already exists in the database.


9

Taylor Swift is much more recent than the works of Stevie Nicks. Let's confirm this by querying music which was released in the '80s, using our new function. We expect both Bella Donna and Tango In The Night to appear in this query, but not Evermore...

In [15]:
formatted_album_between_years(1980, 1989)

[('Tango In The Night', 'Fleetwood Mac', 1987, 'vinyl'),
 ('Greatest Hits', 'Fleetwood Mac', 1988, 'vinyl'),
 ('Tango In The Night', 'Fleetwood Mac', 1987, 'CD'),
 ('Bella Donna', 'Stevie Nicks', 1981, 'vinyl')]

# Artist Popularity

Another development that we can add to this project is some indication of artist popularity among my collection. Which artists do I own more of?

In [16]:
def descending_count_albums_by_artist():
    """This function returns the number of albums I have in my posession by each artist, then sorts by popularity in descending order."""
    cursor.execute("""
        SELECT Artists.name, COUNT(Albums.id) as album_count
        FROM Albums
        JOIN Artists on Albums.artist_id = Artists.id
        GROUP BY Artists.name
        ORDER BY album_count DESC
        """)
    descending_count_query = cursor.fetchall()
    return descending_count_query

def artist_album_count(artist_name):
    """How many albums do I posess from this specific artist?"""
    cursor.execute("""
        SELECT COUNT(Albums.id)
        FROM Albums
        JOIN Artists on Albums.artist_id = Artists.id
        WHERE Artists.name = ?""", (artist_name,))
    count = cursor.fetchone()[0]
    return count

To give a bit more comparison, let's add a few more albums by some of the artists we have been working with...

In [17]:
insert_album('Greatest Hits', 'Fleetwood Mac', 1988)
insert_formatted_album('Greatest Hits', 'vinyl')

insert_album('Penguin', 'Fleetwood Mac', 1973)
insert_formatted_album('Penguin', 'vinyl')

insert_formatted_album('Tango In The Night', 'CD') # This is my favorite Fleetwood Mac album. I own it in both vinyl and CD!

Album 'Greatest Hits' by 'Fleetwood Mac' already exists in the database.
Album 'Penguin' by 'Fleetwood Mac' already exists in the database.


9

Now lets test our above functions. We expect Fleetwood Mac to be in the lead, as we have inserted 3 physical pieces of media by them into the database. 

In [18]:
descending_count_albums_by_artist()

[('Fleetwood Mac', 3), ('Taylor Swift', 1), ('Stevie Nicks', 1)]

In [19]:
artist_album_count('Fleetwood Mac')

3

# Adding Genres

I would additionally enjoy being able to classify the albums in this database by genre(s). What types of music do I usually listen to? By understanding more about my preferences, this will later allow us more ways to select and rediscover my music. 

Let's start this addition to the project by introducing a Genres table to store all possible genres, as well as an AlbumGenres table that maps Albums to Genres.

In [20]:
cursor.execute("""
        CREATE TABLE IF NOT EXISTS Genres (
               id INTEGER PRIMARY KEY,
               genre_name TEXT UNIQUE NOT NULL
        )""")

cursor.execute("""
        CREATE TABLE IF NOT EXISTS AlbumGenres (
               album_id INTEGER,
               genre_id INTEGER,
               PRIMARY KEY (album_id, genre_id)
               FOREIGN KEY (album_id) REFERENCES Albums(id),
               FOREIGN KEY (genre_id) REFERENCES Genres(id)

        )""")

We additionally need functions to insert genres into our genre table, as well as to associate albums to various genres.

In [21]:
def insert_genre(genre_name):
    """Insert a new genre into Genres table, if it doesn't already exist."""
    cursor.execute("""INSERT OR IGNORE INTO Genres (genre_name) VALUES (?)""", (genre_name,))
    conn.commit()
    return cursor.lastrowid

def find_genre_id(genre_name):
    """Retrieve the id of a genre by its name."""
    cursor.execute("""SELECT id FROM Genres WHERE genre_name = ?""", (genre_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None

def associate_album_with_genres(album_title, genre_names):
    """Associate an album with one or more genres."""
    album_id = find_album_id(album_title)
    for genre_name in genre_names:
        genre_id = find_genre_id(genre_name)
        if genre_id is None:
            genre_id = insert_genre(genre_name)
        try:
            cursor.execute("""INSERT INTO AlbumGenres (album_id, genre_id) VALUES (?, ?)""", (album_id, genre_id))
        except sqlite3.IntegrityError:
            # Handle the case where the association already exists
            print(f"Album '{album_title}' is already associated with genre '{genre_name}'.")
    conn.commit()

Functions aiding in query based upon genre will also be helpful...

In [22]:
def get_albums_by_genre(genre_name):
    """Retrieve albums that belong to a specific genre."""
    cursor.execute("""
        SELECT Albums.title, Artists.name, Albums.year
        FROM Albums
        JOIN Artists ON Albums.artist_id = Artists.id
        JOIN AlbumGenres ON Albums.id = AlbumGenres.album_id
        JOIN Genres ON AlbumGenres.genre_id = Genres.id
        WHERE Genres.genre_name = ?
    """, (genre_name,))
    albums = cursor.fetchall()
    return albums

Let's utilize our new genre-related functions with a few of the records we were previously working with.

In [23]:
insert_genre('Rock')
insert_genre('Folk')
insert_genre('Pop')
insert_genre('Alternative')

associate_album_with_genres('Tango In The Night', ['Rock', 'Pop'])
associate_album_with_genres('Evermore', ['Folk', 'Pop'])
associate_album_with_genres('Greatest Hits', ['Dance', 'Rock', 'Pop'])

Album 'Tango In The Night' is already associated with genre 'Rock'.
Album 'Tango In The Night' is already associated with genre 'Pop'.
Album 'Evermore' is already associated with genre 'Folk'.
Album 'Evermore' is already associated with genre 'Pop'.
Album 'Greatest Hits' is already associated with genre 'Dance'.
Album 'Greatest Hits' is already associated with genre 'Rock'.
Album 'Greatest Hits' is already associated with genre 'Pop'.


In [24]:
rock_albums = get_albums_by_genre('Rock')
rock_albums

[('Tango In The Night', 'Fleetwood Mac', 1987),
 ('Greatest Hits', 'Fleetwood Mac', 1988)]

In [25]:
pop_albums = get_albums_by_genre('Pop')
pop_albums

[('Tango In The Night', 'Fleetwood Mac', 1987),
 ('Evermore', 'Taylor Swift', 2020),
 ('Greatest Hits', 'Fleetwood Mac', 1988)]

Within this project I have certainly strengthened my foundational SQL skills through the creation of multiple related tables, as well as by developing functions which interact with the insertion and querying of these tables. I have also extended the project through a few iterations, such as by querying based upon release date, artist popularity, and additionally by adding consideration of album genres.

This file contains many functions which I will be able to work with in order to categorize and explore my own record collection. My next step is to input my own physical media. After that, I intend to explore relationships in my own personal music collection with the functions I have developed above, as well as to potentially brainstorm other innovative ways to aid my music-selection process. Cheers!